[Dataset](https://www.aminer.org/citation)

In [ ]:
import os

def preprocessing(filename = "dblpv13.json"):
  '''
    將NumberInt(N)置換成整數N

    TODO: 111/9/6 bigjson替換成ijson
  '''

  with open(GDRIVE_PATH + filename, "r") as f:
    with open(GDRIVE_PATH + "output", "w") as outputFile:
      while True:
        content = f.readline()
        if content == "": 
          break

        erasePosStart = content.find("NumberInt(")
        if erasePosStart != -1:
          erasePosEnd = content.find(")", erasePosStart)
          content = content[:erasePosStart] + \
                content[erasePosStart+10 : erasePosEnd] + \
                content[erasePosEnd+1:]

        outputFile.write(content)
  try:
    f = open(GDRIVE_PATH + "output", "rb")
    jsonFormat = bigjson.load(f)
    size = len(jsonFormat)
  except Exception as e:
    print(str(e))
    return
  else:
    os.remove(GDRIVE_PATH + filename)
    os.rename(GDRIVE_PATH + "output", GDRIVE_PATH + filename)


In [ ]:
import ijson
from os.path import exists

def captureNodes():
  '''
  從原始資料集中擷取authers的id作為節點，並且排除掉重複後輸出至node
  '''
  nodes = set()
  with open(GDRIVE_PATH + "dblp.json", "rb") as f:
    authors = ijson.items(f, "item")
    author = ""
    try:
      for author in authors:
        if "_id" in author:
          nodes.add(author["_id"])
    except Exception as e:
      print(author)
          
  with open(GDRIVE_PATH + "nodes", "w") as output:
    for node in nodes:
      output.write("\"" + node + "\"\n")

def captureEdges():
  '''
  若任兩個節點只要有一篇共同著作，則視為有邊連在一起
  '''
  output = open(GDRIVE_PATH + "edges", "w")
  with open(GDRIVE_PATH + "dblp.json", "rb") as f:
    parser = ijson.parse(f)
    for prefix, event, value in parser:
      if prefix == "item.authors" and event == "start_array":
        authorsInSamePaper = set()
        while event != "end_array":
          prefix, event, value = next(parser)
          if prefix == "item.authors.item._id":
            authorsInSamePaper.add(value)
            
        for c in combinations(authorsInSamePaper, 2):
          output.write(c[0] + "," + c[1] + "\n")
          

# def accumulateDegrees(bucket: set, nodeID: str):
#   if nodeID not in bucket:
#     bucket[nodeID] = 1
#   else:
#     bucket[nodeID] += 1

# def statisticizeDegrees():
#   # ERROR!! if degree of a node is zero, this node isn't appended.
#   if not exists(GDRIVE_PATH + "edges"):
#     raise Exception("Check both of the files(node and edges) is exist.")

#   degrees = dict()

#   with open(GDRIVE_PATH + "edges", "r") as input:
#     for line in input:
#       src, dit = line.split(",")
#       accumulateDegrees(degrees, src)
#       accumulateDegrees(degrees, dit[:-1]) # remove newline
#   sortedDegrees = {node: degree for node, degree in 
#                    sorted(degrees.items(), key=lambda item: item[1], reverse=True)}
#   return sortedDegrees
  

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules
GDRIVE_PATH = ""

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/dirve")
    GDRIVE_PATH = r"/content/dirve/MyDrive/研究所/Data/dblp/"
    sys.path.append('/content/dirve/MyDrive/Colab Notebooks/package')
else:
    GDRIVE_PATH = r"D:\\論文實驗\\data\\dblp\\"
    sys.path.append('D:\\論文實驗\\package')
    sys.path.append("D:\\論文實驗\\thesis\\Lib\\site-packages")

In [2]:
import networkx as nx 
import unittest

def suite():
    return unittest.defaultTestLoader.discover("./test/")
    
if __name__ == "__main__":
    
    testRunner = unittest.TextTestRunner()
    testRunner.run(suite())
    
    # graph = SN_Graph()
    # graph.construct(GDRIVE_PATH + "edges")

    # graph = SN_Graph()
    # nx.add_path(graph, [0,1,2,3,1], weight=1)
    # nx.add_path(graph, [3,2], weight = 0.7)



......
----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


In [5]:
graph.random_topic(5)
print(nx.get_node_attributes(graph, "topic"))

NameError: name 'graph' is not defined

In [ ]:
list(graph.in_degree)[:10]

[('53f45728dabfaec09f209538', 10),
 ('5601754345cedb3395e59457', 1),
 ('53f43d25dabfaeecd6995149', 2),
 ('5601754345cedb3395e5945a', 3),
 ('53f38438dabfae4b34a08928', 17),
 ('54328883dabfaeb4c6a8a699', 6),
 ('53f46797dabfaeb22f542630', 5),
 ('53f43b03dabfaedce555bf2a', 0),
 ('53f45ee9dabfaee43ecda842', 3),
 ('53f42e8cdabfaee1c0a4274e', 2)]

In [ ]:
subgraph = graph.sampling_subgraph(10)

'2.6.3'

In [ ]:
print(subgraph.number_of_edges())
print(subgraph.size())
print(subgraph.edges)

1818152
1818152


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(subgraph.number_of_nodes())
print(len(subgraph))
print(subgraph.nodes)

710064
710064


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(subgraph.degree(list(subgraph.nodes)))

[('542ab4f0dabfae2b4e134367', 0), ('544082a6dabfae805a6c8d50', 0), ('5606ebca45cedb3396996b4a', 0), ('53f44d3bdabfaee43ec98f1a', 0), ('53f431dddabfaee0d9b35ef7', 0), ('53f439c9dabfaedd74dbfca0', 0), ('53f44001dabfaeee229e69b6', 0), ('53f42eb6dabfaee2a1c8d9dc', 0), ('53f42d57dabfaee02ac605d0', 0), ('5489b9c3dabfae9b40134c9a', 0)]


![image](https://cdn.discordapp.com/attachments/498518865802952706/1019575970103050240/unknown.png)

In [ ]:
numEdges = 0
with open(GDRIVE_PATH + "edges", "r") as f:
  for line in f:
    numEdges += 1
print("number of edges:" + str(numEdges))

numNodes = 0
with open(GDRIVE_PATH + "nodes", "r") as f:
  for line in f:
    numNodes += 1
print("number of nodes:" + str(numNodes))

number of edges:2185408
number of nodes:834826


In [ ]:
with open(GDRIVE_PATH + "degrees", "r") as f:
  for line in range(100):
    print(f.readline(), end="")

In [ ]:
with open(GDRIVE_PATH + "dblp.json", "r") as f:
  for line in range(100):
    print(f.readline(), end="")

[
{ 
    "_id" : "53e99784b7602d9701f3e3f5", 
    "title" : "3GIO.", 
    "venue" : {
        "type" : 0
    }, 
    "year" : 2011, 
    "keywords" : [

    ], 
    "n_citation" : 0, 
    "lang" : "en"
},
{ 
    "_id" : "53e99784b7602d9701f3e133", 
    "title" : "The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.", 
    "authors" : [
        {
            "_id" : "53f45728dabfaec09f209538", 
            "name" : "Peijuan Wang"
        }, 
        {
            "_id" : "5601754345cedb3395e59457", 
            "name" : "Jiahua Zhang"
        }, 
        {
            "_id" : "53f38438dabfae4b34a08928", 
            "name" : "Donghui Xie"
        }, 
        {
            "_id" : "5601754345cedb3395e5945a", 
            "name" : "Yanyan Xu"
        }, 
        {
            "_id" : "53f43d25dabfaeecd6995149", 
            "name" : "Yun Xu"
        }
    ], 
    "venue" : {
        "_id" : "53a7297d20f7420be8bd4ae7", 
    

In [9]:
import sys
sys.path

['D:\\論文實驗',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 '/package',
 '/package',
 '/thesis/Lib/site-packages',
 'D:\\論文實驗\\thesis\\Lib\\site-packages']